In [1]:
using LinearAlgebra
using IterTools
using NPZ
using FFTW
using Base.Threads
using PyPlot

println(Threads.nthreads())
# println(pwd())

54
/home/wang/Documents/HEB/pf_distort/msad_microelasticity


Interaction kernel generating program

In [ ]:
function green_k(k_vec, c_ij, ijkl2ij_dict)
    k_norm = k_vec/norm(k_vec)
    zij = Complex.(zeros(3,3))
    for (i, k) in product(1:3, 1:3)
        for (j, l) in product(1:3, 1:3)
            ji_denote = ijkl2ij_dict[(j,i)]
            kl_denote = ijkl2ij_dict[(k,l)]
            c_voigt = c_ij[ji_denote, kl_denote]
            zij[i,k] += c_voigt * k_norm[j] * k_norm[l]
        end
    end

    zij_inv = transpose(inv(zij))
    return zij_inv/norm(k_vec)^2
end

function prefac_tensor(k_vec, c_ij, ijkl2ij_dict)

    if norm(k_vec) == 0
        return Complex.(zeros(3))
    end

    g_tensor = green_k(k_vec, c_ij, ijkl2ij_dict)
    prefac_list = Complex.(zeros(3))

    for i in 1:3
        for (j,l,m,n) in product(1:3, 1:3, 1:3, 1:3)

            jl_denote = ijkl2ij_dict[(j,l)]
            mn_denote = ijkl2ij_dict[(m,n)]
            c_voijt = c_ij[jl_denote, mn_denote]
            kl = k_vec[l]
            gij = g_tensor[i,j]

            prefac_list[i] += -im * c_voijt * kl * gij

        end
    end

    return prefac_list 
end

#* some constants
mjm2eva = 6.24150965*1e-5 #* convert mJ/m^2 to eV/Å^2
eva2gpa = 160.2176621 #* convert eV/Å^3 to GPa
specie_denote = "50Ti50ZrB2" #TODO specie denotion

#* lattice initialize
nx, ny, nz = 32,32,32 #TODO nx*ny*nz prim cells
a, c = 3.11, 3.39 #TODO lattice constant

#TODO cij part (1120-0001-1100)
cij = npzread("dataset/cij/TiZrB_cij_$(a)_$(c)_5005000_try_1_fin.npy")

ijkl2ij_dict = Dict(
    (1,1)=>1, (2,2)=>2, (3,3)=>3,
    (2,3)=>4, (1,3)=>5, (1,2)=>6,
    (2,1)=>6, (3,1)=>5, (3,2)=>4)
cij = cij/eva2gpa #* convert to eV/Å^3

#* grid sampling based on atomic spacing
lx, ly, lz = a*nx, c*ny, sqrt(3)*a*nz
len_nx, len_ny, len_nz = nx, ny, nz 
#* initialize frequency grid 
len_kx, len_ky, len_kz = len_nx, len_ny, len_nz
kx = fftfreq(len_kx, len_nx/lx)*2*pi 
ky = fftfreq(len_ky, len_ny/ly)*2*pi
kz = fftfreq(len_kz, len_nz/lz)*2*pi
dkx = kx[2] - kx[1]
dkz = kz[2] - kz[1] 
dky = ky[2] - ky[1]
dxyz = dkx*dky*dkz
len_kxy = len_kx*len_ky

kz_sample = length(kz)
kz = Complex.(kz)
prefac_mesh = Complex.(zeros((len_kxy*kz_sample, 3)))
prod_kxky = collect(product(1:len_kx, 1:len_ky))
kxyz_mesh = (zeros((len_kxy*kz_sample, 3)))

Threads.@threads for kxy_i in 1:len_kxy 

    ind_kx, ind_ky = prod_kxky[kxy_i]
    kx_val, ky_val = kx[ind_kx], ky[ind_ky]

    for kz_i in 1:kz_sample
        k_vec = [kx_val, ky_val, kz[kz_i]]
        prefac = prefac_tensor(k_vec, cij, ijkl2ij_dict)
        prefac_mesh[(kxy_i-1)*kz_sample+kz_i,:] .= prefac
        kxyz_mesh[(kxy_i-1)*kz_sample+kz_i,:] .= k_vec
    end 
end

prefac_1, prefac_2, prefac_3 = prefac_mesh[:,1], prefac_mesh[:,2], prefac_mesh[:,3]

(ComplexF64[0.0 + 0.0im, 0.0 - 54.86825167237674im, 0.0 - 27.43412583618837im, 0.0 - 18.289417224125582im, 0.0 - 13.717062918094186im, 0.0 - 10.973650334475348im, 0.0 - 9.144708612062791im, 0.0 - 7.838321667482393im, 0.0 - 6.858531459047093im, 0.0 - 6.09647240804186im  …  0.0 + 6.217121518258872im, 0.0 + 6.9057004290404365im, 0.0 + 7.740469586557749im, 0.0 + 8.761372565058062im, 0.0 + 10.0136544130905im, 0.0 + 11.531003830677841im, 0.0 + 13.27508483923075im, 0.0 + 14.965939043975169im, 0.0 + 15.84478830850127im, 0.0 + 15.519682841370566im], ComplexF64[0.0 + 0.0im, 0.0 - 54.86825167237675im, 0.0 - 27.434125836188375im, 0.0 - 18.289417224125582im, 0.0 - 13.717062918094188im, 0.0 - 10.973650334475346im, 0.0 - 9.144708612062791im, 0.0 - 7.838321667482393im, 0.0 - 6.858531459047094im, 0.0 - 6.09647240804186im  …  0.0 + 6.163846497831909im, 0.0 + 6.853594709749645im, 0.0 + 7.695176202475175im, 0.0 + 8.734396432590609im, 0.0 + 10.028441809145654im, 0.0 + 11.635410007667215im, 0.0 + 13.5644819

In [38]:
try
    mkdir("kernel/runs/$(specie_denote)")
catch e
    println("directory already exists") 
end

sav_pth = "kernel/runs/$(specie_denote)/$(specie_denote)_$(a)_$(c)_$(nx)_$(ny)_$(nz)"
npzwrite(sav_pth*"_kmesh.npy", kxyz_mesh[2:end, :])

npzwrite(sav_pth*"_lambda1.npy", prefac_1[2:end, :])
npzwrite(sav_pth*"_lambda2.npy", prefac_2[2:end, :])
npzwrite(sav_pth*"_lambda3.npy", prefac_3[2:end, :])